In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews import opts
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_market_data)

In [ ]:
# Create a DataFrame with the scaled data
columns = df_market_data.columns
df_scaled = pd.DataFrame(scaled_data, columns=columns)


# Copy the crypto names from the original data
crypto_names = df_market_data.index

# Set the coinid column as index
df_scaled['coin_id'] = crypto_names
df_scaled.set_index('coin_id', inplace=True)

# Display sample data
print(df_scaled.head())

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))


In [ ]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
 kmeans_model = KMeans(n_clusters=k, random_state=42)
# 2. Fit the model to the data using `df_market_data_scaled`
kmeans_model.fit(df_scaled)
# 3. Append the model.inertia_ to the inertia list
 inertia_values.append(kmeans_model.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k_values': k_values,
    'inertia_values': inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
plt.plot(df_elbow['k_values'], df_elbow['inertia_values'], marker='o')
plt.title('Elbow Curve to Find Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.show()

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialise the K-Means model using the best value for k
best_k = 11
kmeans_model = KMeans(n_clusters=best_k, random_state=42)

In [ ]:
# Fit the K-Means model using the scaled data
kmeans_model.fit(df_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
clusters = kmeans_model.predict(df_scaled)


# Print the resulting array of cluster values.
print("Cluster Values:", clusters)

In [ ]:
# Create a copy of the DataFrame
df_clustered = df_scaled.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_clustered['cluster'] = clusters

# Display sample data
print(df_clustered.head())

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Colour the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_clustered.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="cluster",
    hover_cols=["coin_id"],
    width=800,
    height=500,
    title="Crypto Clusters",
    xlabel="24h Price Change Percentage",
    ylabel="7d Price Change Percentage"
)

---

### Optimise Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to three principal components.
principal_components = pca.fit_transform(df_scaled)
df_pca = pd.DataFrame(
    data=principal_components,
    columns=['Principal Component 1', 'Principal Component 2', 'Principal Component 3'],
    index=df_scaled.index
)

# View the first five rows of the DataFrame. 
print(df_pca.head())




In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_
print("Explained Variance:")
for i, variance in enumerate(explained_variance):
    print(f"Principal Component {i+1}: {variance:.2%}")

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [ ]:
# Create a new DataFrame with the PCA data.
# Creating a DataFrame with the PCA data
df_pca_result = pd.DataFrame(
    data=principal_components,
    columns=['Principal Component 1', 'Principal Component 2', 'Principal Component 3'],
    index=df_scaled.index
)

# Copy the crypto names from the original data
crypto_names_pca = df_scaled.index

# Set the coinid column as index
df_pca_result['coin_id'] = crypto_names_pca
df_pca_result.set_index('coin_id', inplace=True)

# Display sample data
print(df_pca_result.head())


---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values_pca = list(range(1, 12))

In [ ]:
# Create an empty list to store the inertia values
inertia_values_pca = []

# Create a for loop to compute the inertia with each possible value of k
for k_pca in k_values_pca:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
kmeans_model_pca = KMeans(n_clusters=k_pca, random_state=42)
# 2. Fit the model to the data using `df_market_data_pca`
kmeans_model_pca.fit(df_pca_result)
# 3. Append the model.inertia_ to the inertia list
inertia_values_pca.append(kmeans_model_pca.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    'k_values': k_values_pca,
    'inertia_values': inertia_values_pca
}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)


In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
plt.plot(df_elbow_pca['k_values'], df_elbow_pca['inertia_values'], marker='o')
plt.title('Elbow Curve for PCA Data')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.show()

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** 

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialise the K-Means model using the best value for k
best_k_pca = 4
kmeans_model_pca = KMeans(n_clusters=best_k_pca, random_state=42)

In [ ]:
# Fit the K-Means model using the PCA data
kmeans_model_pca.fit(df_pca_result)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
clusters_pca = kmeans_model_pca.predict(df_pca_result)
# Print the resulting array of cluster values.
print("Cluster Values (PCA):", clusters_pca)

In [ ]:
# Create a copy of the DataFrame with the PCA data
df_clustered_pca = df_pca_result.copy()


# Add a new column to the DataFrame with the predicted clusters
df_clustered_pca['cluster_pca'] = clusters_pca

# Display sample data
print(df_clustered_pca.head())

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Colour the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_clustered_pca.hvplot.scatter(
    x="Principal Component 1",
    y="Principal Component 2",
    by="cluster_pca",
    hover_cols=["coin_id"],
    width=800,
    height=500,
    title="Crypto Clusters with PCA",
    xlabel="Principal Component 1",
    ylabel="Principal Component 2"
)

### Visualise and Compare the Results

In this section, you will visually analyse the cluster analysis results by contrasting the outcome with and without using the optimisation techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
elbow_curve = df_elbow_pca.hvplot.line(
    x='k_values',
    y='inertia_values',
    title='Elbow Curve for PCA Data',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    width=400,
    height=300
)

In [ ]:
# Composite plot to contrast the clusters
cluster_scatter = df_clustered_pca.hvplot.scatter(
    x="Principal Component 1",
    y="Principal Component 2",
    by="cluster_pca",
    hover_cols=["coin_id"],
    width=800,
    height=500,
    title="Crypto Clusters with PCA",
    xlabel="Principal Component 1",
    ylabel="Principal Component 2"
)

(elbow_curve + cluster_scatter).opts(
    opts.Layout(sublabel_format="{label}").cols(2)
)

#### Answer the following question: 

  * **Question:** After visually analysing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 